### Monadic function `⍸`
*Where* returns the indices of the 1s in the Boolean argument:

In [1]:
⍸1 0 1 1 0 0 1

1 3 4 7

In [4]:
M←2 3⍴1 0 1 1
M
⍸M

1 0 1
1 1 0

┌───┬───┬───┬───┐
│1 1│1 3│2 1│2 2│
└───┴───┴───┴───┘

### Dyadic function `⍸`
*Interval index* takes a list of sorted arrays on the left, and for each array on the right, tells which "gap" (interval) it belongs to:

In [5]:
1 10 100 1000⍸0 500 2000 3 10 

0 3 4 1 2

So 0 is in interval number 0 (that is, before 1–10). 500 is in interval 3, which is 100–1000, etc. 

### Monadic function `⊆`
This one is called *Nest* because it guarantees you that the result is nested (non-simple).
`(1 2)(3 4 5)` is already nested, so `⊆` won't do anything: 

In [7]:
(1 2)(3 4 5)
⊆(1 2)(3 4 5)

┌───┬─────┐
│1 2│3 4 5│
└───┴─────┘

┌───┬─────┐
│1 2│3 4 5│
└───┴─────┘

`1 2 3` is not nested, so `⊆` will nest it:

In [9]:
1 2 3
⊆1 2 3

1 2 3

┌─────┐
│1 2 3│
└─────┘

### Dyadic function `⊆`
*Partition* was formerly only available as `{⎕ML←3 ⋄ ⍺⊂⍵}`. It partitions its right argument letting a new partition begin whenever an element is higher than its neighbour in the left. Also, elements indicated by 0s are dropped completely:

In [10]:
1 0 0 1 1 3 2 2 5 5 0⊆'Hello World'

┌─┬──┬───┬──┐
│H│lo│ Wo│rl│
└─┴──┴───┴──┘

### Dyadic operator `@`
The *At* operator does exactly what it says. What's on its left gets done **at** the position indicated by its right operand:

In [11]:
('X'@2 5) 'Hello'

HXllX

We can also give an array which matches the selected elements:

In [12]:
('XY'@2 5) 'Hello'

HXllY

We can also use it to modify rather than replace elements:

In [13]:
(-@2 5) 10 20 30 40 50 60

10 ¯20 30 40 ¯50 60

In [15]:
7 (+@2 5) 10 20 30 40 50 60

10 27 30 40 57 60

If we use a function right operand, it gets applied to the right argument, and the result must be a Boolean mask which then used instead of the list of indices.

In [20]:
(∊∘⎕A) 'Hello World'  ⍝ ⎕A is the uppercase alphabet

1 0 0 0 0 0 1 0 0 0 0

In [21]:
'x'@(∊∘⎕A) 'Hello World'  ⍝ replace those

xello xorld

### Dyadic operator `⌺`
*Stencil* (as in [stencil code](https://en.wikipedia.org/wiki/Stencil_code)) and the symbol is supposed to evoke the picture of a stencil over a paper. It takes a function left operand and an array right operand, and derives a monadic function.

The right operand specifies what neighbourhoods to apply to. E.g. in Game of Life, the neighbourhoods are 3-by-3 sub-matrices centred on each element in the input array. The function operand gets called dyadically with the right argument being a neighbourhood and the argument left being information about whether he neighbourhood overlaps an edge of the original argument world.

To see how it works, we'll use `{⊂⍵}` as left operand. It just encloses the right argument (`⍵`, that is, the neighbourhood) so we can see it. As right operand (neighbourhood size) we use `3 3`.

In [22]:
⎕←4 6⍴⎕A  ⍝ our world

ABCDEF
GHIJKL
MNOPQR
STUVWX

In [23]:
({⊂⍵}⌺3 3) 4 6⍴⎕A  ⍝ the neighbourhoods

┌───┬───┬───┬───┬───┬───┐
│ │ │ │ │ │ │
│ AB│ABC│BCD│CDE│DEF│EF │
│ GH│GHI│HIJ│IJK│JKL│KL │
├───┼───┼───┼───┼───┼───┤
│ AB│ABC│BCD│CDE│DEF│EF │
│ GH│GHI│HIJ│IJK│JKL│KL │
│ MN│MNO│NOP│OPQ│PQR│QR │
├───┼───┼───┼───┼───┼───┤
│ GH│GHI│HIJ│IJK│JKL│KL │
│ MN│MNO│NOP│OPQ│PQR│QR │
│ ST│STU│TUV│UVW│VWX│WX │
├───┼───┼───┼───┼───┼───┤
│ MN│MNO│NOP│OPQ│PQR│QR │
│ ST│STU│TUV│UVW│VWX│WX │
│ │ │ │ │ │ │
└───┴───┴───┴───┴───┴───┘

We got a 4-by-6 matrix of neighbourhoods back. Notice that all the neighbourhoods are 3-by-3, even at the edges, at which they were padded with spaces. The padding was done sometimes on top, sometimes on the left, sometimes on the right, and sometimes on the bottom. The information about that is in the left argument (`⍺`) of the operand function:

In [24]:
({⊂⍺}⌺3 3) 4 6⍴⎕A

┌────┬────┬────┬────┬────┬─────┐
│1 1 │1 0 │1 0 │1 0 │1 0 │1 ¯1 │
├────┼────┼────┼────┼────┼─────┤
│0 1 │0 0 │0 0 │0 0 │0 0 │0 ¯1 │
├────┼────┼────┼────┼────┼─────┤
│0 1 │0 0 │0 0 │0 0 │0 0 │0 ¯1 │
├────┼────┼────┼────┼────┼─────┤
│¯1 1│¯1 0│¯1 0│¯1 0│¯1 0│¯1 ¯1│
└────┴────┴────┴────┴────┴─────┘

So, each cell contains two elements, one for rows, and one for columns. Positive indicates left/top. Negative is right/bottom. The magnitude indicates how many rows/columns were padded. This fits nicely with dyadic `↓` (*Drop*), which as left argument takes the number of (rows,columns) to drop from its right argument:

In [25]:
({⊂⍺↓⍵}⌺3 3) 4 6⍴⎕A

┌──┬───┬───┬───┬───┬──┐
│AB│ABC│BCD│CDE│DEF│EF│
│GH│GHI│HIJ│IJK│JKL│KL│
├──┼───┼───┼───┼───┼──┤
│AB│ABC│BCD│CDE│DEF│EF│
│GH│GHI│HIJ│IJK│JKL│KL│
│MN│MNO│NOP│OPQ│PQR│QR│
├──┼───┼───┼───┼───┼──┤
│GH│GHI│HIJ│IJK│JKL│KL│
│MN│MNO│NOP│OPQ│PQR│QR│
│ST│STU│TUV│UVW│VWX│WX│
├──┼───┼───┼───┼───┼──┤
│MN│MNO│NOP│OPQ│PQR│QR│
│ST│STU│TUV│UVW│VWX│WX│
└──┴───┴───┴───┴───┴──┘